<a href="https://colab.research.google.com/github/nokomoro3/book-ml-transformers/blob/main/ml-transformers-chap04-multilingal-ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
from dateutil import tz

start_datetime = datetime.datetime.now().astimezone(tz.gettz('Asia/Tokyo'))
print(f"start  : {start_datetime.isoformat()}")

start  : 2022-09-04T11:27:33.953951+09:00


- System Specs
  - https://colab.research.google.com/drive/151805XTDg--dgHb3-AXJCpnWaqRhop_2

In [2]:
!cat /etc/os-release | grep "PRETTY_NAME" | awk -F"=" '{print $2}'

"Ubuntu 18.04.6 LTS"


In [3]:
!!nvidia-smi -L

['GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-3a586ff5-2139-32ab-8627-09659cf06d43)']

In [4]:
!free -h --si | awk  '/Mem:/{print $2}'

26G


In [ ]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

Cloning into 'notebooks'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 422 (delta 72), reused 68 (delta 68), pack-reused 343
Receiving objects: 100% (422/422), 25.01 MiB | 26.93 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [ ]:
#hide
from utils import *
setup_chapter()

Using transformers v4.11.3
Using datasets v1.16.1


# テキスト生成

- GPT-2やその後継のGPT-3は与えられたテキストの続きとなるリアルなテキストを生成できる。
- これらにより、会話エージェントなど様々な応用が生まれている。
- 本章ではGPT-2を使って、テキスト生成の仕組みや戦略の違いによる生成物の違いについて説明する。

## 5.1 一貫した内容のテキストを生成するための課題

- これまで扱ってきた系列とトークン分類は予測確率を得るヘッドを付与するのみで実現できる。
- 一方テキスト生成の場合は、モデルの確率的出力をテキスト変換するために、デコード手法が必要となる。
- これは以下のような課題を生み出す。
  - デコードは反復的に行う必要があるため、計算量が大きくなる。
  - 生成されるテキストの品質と多様性は、デコード手法やハイパーパラメータに依存する。
- まずはデコードの仕組みを理解するために、GPT-2の事前学習と後段のテキスト生成への適用を確認する。
- GPT-2はトークン列 $ \bm{y}=y_1,y_2,y_3,...y_t $ の確率 $ P(\bm{y}|\bm{x}) $ を推定するよう事前学習される。
  - これは、自己回帰言語モデル(autogressive)や因果的言語モデル(causal language model)と同様である。
  - ここで $ \bm{x} $ は与えられる初期の文脈列 $ \bm{x} = x_1,x_2,x_3,...x_k $ である。
- $ \bm{x} $から直接 $ P(\bm{y}|\bm{x}) $ を推定するのに十分な学習データを得るのは現実的ではないため、以下のような条件付確率の積に分解する。

$$ P(y_1,y_2,y_3,...y_t|\bm{x}) = \prod^{N}_{t=1}{P(y_t|y_{<t},\bm{x}}) $$

- ここで、 $ y_{<t} $ は $ y_1,...,y_{t-1} $ の略記法である。
- この式から直観的に出現済みの単語から各単語を予測すると見なすことができる。
- マスクされたトークンを予測するために前後の文脈を利用するBERTとは全く異なる構造となっている。
- 図で表現すると以下のような形で順次処理される。

![](./img/ml-transformers-chap05-text-generation_2022-09-04-19-40-13.png)

- これを、EOSトークンが出力されるか、事前に定義した最大長まで繰り返し行う。
- 各時点での処理は、どのトークンを選択するかを決定するデコード処理が適用される。
- 実際には、以下のようにトークン毎のロジット、$ z_{t,i} $ をモデルが出力するので、softmaxを適用して、次のトークン $ w_{i} $を得る。

$$ P(y_t = w_t | y_{<t}, \bm{x}) = \text{softmax}(z_{t,i}) $$

- 最終的な目標は最も可能性の高い系列 $ \bm{\hat{y}} $ を探索することである。

$$ \bm{\hat{y}} = \argmax_{\bm{y}}{P(\bm{y}|\bm{x})}$$

-  $ \bm{\hat{y}} $ を見つけるには、可能な限りのパターンの系列を調べる必要があるが、現実では実行が難しいため近似手法が用いられる。
-  本章では、これらの近似手法をいくつか調査し、よりスマートで複雑なアルゴリズムを導出する。


## 5.2 貪欲法によるデコード

- WIP